<a href="https://colab.research.google.com/github/ErikHerazo/image_clasifier/blob/main/image_classifier_end.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/pytorch/vision.git

In [ ]:
# importacion de librerias

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(4):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.083
[1,  4000] loss: 1.089
[1,  6000] loss: 1.068
[1,  8000] loss: 1.106
[1, 10000] loss: 1.078
[1, 12000] loss: 1.072
[2,  2000] loss: 1.009
[2,  4000] loss: 1.011
[2,  6000] loss: 1.024
[2,  8000] loss: 1.053
[2, 10000] loss: 1.048
[2, 12000] loss: 1.039
[3,  2000] loss: 0.956
[3,  4000] loss: 0.955
[3,  6000] loss: 0.994
[3,  8000] loss: 0.975
[3, 10000] loss: 1.009
[3, 12000] loss: 0.998
[4,  2000] loss: 0.920
[4,  4000] loss: 0.933
[4,  6000] loss: 0.938
[4,  8000] loss: 0.960
[4, 10000] loss: 0.952
[4, 12000] loss: 0.980
Finished Training


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')


Accuracy of the network on the 10000 test images: 60 %


In [ ]:
from PIL import Image

# Función para cargar y preprocesar la imagen
def process_image(image_path):
    image = Image.open(image_path)
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    image = transform(image).unsqueeze(0)  # Agrega una dimensión de batch
    return image

# Cargar la imagen y predecir la clase
def predict_image_class(image_path, model):
    image = process_image(image_path)
    model.eval()  # Poner el modelo en modo de evaluación
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        return classes[predicted[0]]

# Predecir la clase de una nueva imagen
image_path = '/content/perro2.jpg' # ruta de la imagen a clasificar
predicted_class = predict_image_class(image_path, net)
print(f'La imagen es clasificada como: {predicted_class}')